# PostgreSQL

In [1]:
import sqlalchemy as db
import zeit
import pandas as pd
import configparser

In [3]:
c = configparser.ConfigParser()
c.read("config.ini")
zeit_key = c.get("Zeit", "key")
uri = c.get("Postgres", "URI")

#set the API, authenticate and check status
api = zeit.API()
api.set_token(zeit_key)
api.get_status()

Exception: Not a good key

In [15]:
#engine = db.create_engine("postgresql://code@localhost/test") #local
engine = db.create_engine(uri) #on herokus
con = engine.connect()
metadata = db.MetaData()

### Retrieving Values

In [16]:
table = db.Table("keyword", metadata, autoload=True, autoload_with=engine)

In [37]:
word = "Merkel"
query = db.select([table])#.where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()
try:
    df = pd.DataFrame(result, columns = result[0].keys())
except Exception:
    print("No Results")

In [38]:
df

,uri,lexical,results,type,json,matches,score
0,http://api.zeit.de/keyword/angela-merkel,Angela Merkel,9418,person,"{""638645094239"":5,""670181094239"":9,""7018034942...",{'subtitle': 'Der Tourismusbeauftragte der Bun...,93
1,http://api.zeit.de/keyword/roettgen,Norbert Röttgen,128,location,"{""1143566695229"":3,""1206725095229"":2,""12382610...",{'subtitle': 'Laut einem Bericht könnte Deutsc...,36
2,http://api.zeit.de/keyword/gazastreifen,gazastreifen,183,subject,"{""1206725096227"":2,""1269797096227"":1,""13329554...",{'subtitle': 'Erstmals seit Jahren ist der Gre...,53
3,http://api.zeit.de/keyword/armin-laschet,Armin Laschet,475,person,"{""859571158011"":1,""1017337558011"":1,""108049595...",{'subtitle': 'Ein Ländervergleich aus gegebene...,9


### Inserting Values

In [17]:
table.columns.keys()

['uri', 'lexical', 'results', 'type', 'json', 'matches', 'score']

In [35]:
r = api.get_keyword("armin-laschet", facet_time="1year")

In [36]:
query = db.insert(table).values(
    uri = r.uri, lexical = r.name, results = r.found, 
    type = r.type, json = r.get_facets().to_json(),
    matches = r.get_raw()["matches"][0], score = r.score
)
ResultProxy = con.execute(query)

### Deleting Values

In [32]:
delete = table.delete().where(table.columns.lexical == 'Laschet, Armin')
ResultProxy = con.execute(delete)

## Workflow for inserting

In [25]:
word_list = ["angela-merkel", "roettgen", "gazastreifen"]
for w in word_list:
    #call the api
    r = api.get_keyword(w, facet_time="1year", limit = 1)

    try:
        matches = r.get_raw()["matches"][0]
    except Exception:
        matches = ''

    #construct the insert query
    query = db.insert(table).values(
        uri = r.uri, lexical = r.name, results = r.found, 
        type = r.type, json = r.get_facets().to_json(),
        matches = matches, score = r.score,
        )
    #execute it
    ResultProxy = con.execute(query)

# Get the Data in Dash

In [ ]:
word = "gazastreifen"
query = db.select([table]).where(table.columns.uri == f"http://api.zeit.de/keyword/{word}")
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()

In [ ]:
#better querys
query = db.select([table]).filter(db.or_( #SELECT * FROM table WHERE 
        table.columns.uri.like(f"%meck%"), #uri LIKE '%word%' OR
        table.columns.lexical.like(f"%meck%"), #lexical LIKE '%word%' OR
        )
        ).order_by(db.desc(table.columns.results)).limit(5) #ORDER BY results DESC LIMIT 5
ResultProxy = con.execute(query)
result = ResultProxy.fetchall()